<a href="https://colab.research.google.com/github/JhonnyLimachi/Sigmoidal/blob/main/52_Limpeza_de_dados%2C_outliers_e_valores_ausentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="center"><img src="https://raw.githubusercontent.com/carlosfab/escola-data-science/master/img/eds.png" height="100px"></p>

# Limpeza de dados, outliers e valores ausentes

Neste projeto, vamos focar em uma habilidade básica, mas essencial de um Cientista de Dados. Saber lidar com valores ausentes, lidar com *outliers*, transformar os dados. Essas são algumas das atividades que fazem parte do checklist de todos os projetos que executamos.


Esses processos podem ter um impacto enorme nos resultados, desde a parte da análise dos dados até os modelos de *Machine Learning*.
<p align="center"><img src="https://image.freepik.com/free-vector/group-analysts-working-graphs_1262-21249.jpg
"></p>


Aqui, veremos o que buscar entender nos dados, os passos a dar, como identificar, e tratar *outliers*, as melhores práticas e métodos para lidar com dados ausentes, e por fim, converter os dados para os formatos necessários.


## O Que Analisar?

Primeiro de tudo, precisamos entender o que são bons dados, para que saibamos os passos que precisamos tomar para ter o melhor conjunto de dados possível, a partir dos dados que estamos utilizando.

* Tipo de Dados
  * Coluna de data em formato `datetime`, coluna de valores monetários em `float`.
* *Range* dos Dados
  * Meses de 1 a 12, dias do mês de 1 a 31.
* Dados Obrigatórios
  * Algumas colunas não podem estar vazias
* Dados Únicos
  * CPF, RG, CNPJ, ID de usuário.
* Dados Categóricos
  * Gênero Masculino ou Feminino.
* Padrões regulares
  * (61) 9 8765-4321
* Validade Entre os Campos
  * Data de saída não ser anterior à data de entrada.
